# Asociados con todas sus marcas. Correr con el Stock de todos los asociados.

In [2]:
# Librerías usadas:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import funciones

In [20]:
# DEFINICION DE LAS RUTAS Y VARIABLES:

# Archivo al que se exportara la informacion:
reporte_consolidado = 'C:/Users/lsaucedo/downloads/reporte_consolidado.csv'

# Carpeta donde se encuentran las auxiliares:
ruta_aux = 'C:/modelos/auxiliares/'
aux_pmo = 'Aux_PMO.xlsx'

# Archivo .csv con los diagnosticos:
ruta_diag = 'C:/Users/lsaucedo/downloads/diagnostico.csv'


# Ruta con Stocks de la zona:
ruta_stock_zona = 'C:/Users/lsaucedo/Desktop/Costo PMO 202112/Stocks_Zona/'
# Ruta con Consumos de la zona:
ruta_consumos_zona = 'C:/Users/lsaucedo/Desktop/Costo PMO 202112/Consumos_Zona/'


In [13]:
# Cargamos y cambiamos los nombres de los stocks.
dfs = funciones.leer_xlsx(ruta_stock_zona)
funciones.renombrar_columnas(dfs)

Inicio del proceso: 14:22:12
Se cargó el archivo:  20.1 Stock Mendoza.xlsx a las: 14:23:39
Fin de la carga de archivos


In [21]:
# Cargamos y cambiamos los nombres de los consumos.
dfc = funciones.leer_xlsx(ruta_consumos_zona)
funciones.renombrar_columnas(dfc)

Inicio del proceso: 14:25:05
Se cargó el archivo:  10.1 Cons Mendoza 202010.xlsx a las: 14:25:59
Se cargó el archivo:  10.1 Cons Mendoza 202011.xlsx a las: 14:26:56
Se cargó el archivo:  10.1 Cons Mendoza 202012.xlsx a las: 14:27:55
Se cargó el archivo:  10.1 Cons Mendoza 202101.xlsx a las: 14:28:52
Se cargó el archivo:  10.1 Cons Mendoza 202102.xlsx a las: 14:29:49
Se cargó el archivo:  10.1 Cons Mendoza 202103.xlsx a las: 14:31:00
Se cargó el archivo:  10.1 Cons Mendoza 202104.xlsx a las: 14:32:38
Se cargó el archivo:  10.1 Cons Mendoza 202105.xlsx a las: 14:34:23
Se cargó el archivo:  10.1 Cons Mendoza 202106.xlsx a las: 14:36:08
Se cargó el archivo:  10.1 Cons Mendoza 202107.xlsx a las: 14:38:00
Se cargó el archivo:  10.1 Cons Mendoza 202108.xlsx a las: 14:39:50
Se cargó el archivo:  10.1 Cons Mendoza 202109.xlsx a las: 14:41:36
Fin de la carga de archivos


In [22]:
# Levantamos los diferentes archivos auxiliares:

# Rango Edad
Aux_RangoEdad = pd.read_excel(ruta_aux + "Aux_RangoEdad.xlsx")
dfs = pd.merge(left = dfs, right = Aux_RangoEdad[["Edad","Rango_Edad"]], left_on='Edad', right_on='Edad', how = "left")
Aux_RangoEdad = []

# Cartilla Base
Aux_CartillaBase = pd.read_excel(ruta_aux+"Aux_CartillaBase.xlsx")
dfs = pd.merge(left = dfs, right = Aux_CartillaBase[["Plan","Modelo_Cartilla"]], left_on='Plan', right_on='Plan', how = "left")
Aux_CartillaBase = []

# Cobertura Medicamentos del plan
Aux_CobertMedicamentosPlan = pd.read_excel(ruta_aux+"Aux_CobertMedicamentosPlan.xlsx")
dfs = pd.merge(left = dfs, right = Aux_CobertMedicamentosPlan[["Plan","Cobertura_med_id"]], left_on='Plan', right_on='Plan', how = "left")
Aux_CobertMedicamentosPlan = []

# Lista de precios Copagos
Aux_ListaCopagos = pd.read_excel(ruta_aux+"Aux_ListaCopagos.xlsx")
dfs = pd.merge(left = dfs, right = Aux_ListaCopagos[["Plan","Lista_PrecioDesc"]], left_on='Plan', right_on='Plan', how = "left")
Aux_ListaCopagos = []

# Levanto el archivo con los diagnosticos:
dfdiag = pd.read_csv(ruta_diag)
dfs = pd.merge(left = dfs, right = dfdiag[["Persona",'GC', 'Rango_Cons', 'algun_diag', 'prop_diag', 'Marca_Diagnostico']], left_on='Persona', right_on='Persona', how = "left")
dfdiag = []

In [24]:
# Reemplazo valores de filas para tener metricas:
dfs['Diabetes'] = dfs['Diabetes'].replace(['No Diabético', 'Diabético'], [0, 1])
dfs['Hepatitis'] = dfs['Hepatitis'].replace(['Sin Hepatitis', 'Con Hepatitis'], [0, 1])
dfs['PMI'] = dfs['PMI'].replace(['NO', 'SI'], [0, 1])
dfs['Cronico'] = dfs['Cronico'].replace(['NO', 'SI'], [0, 1])
dfs['Oncologico'] = dfs['Oncologico'].replace(['Sin Marca', 'Paciente Oncológico'], [0, 1])
dfs['algun_diag'] = dfs['Marca_Diagnostico'].replace(['Sin Diagnostico', 'Con Diagnostico'], [0, 1])

# Reemplazo los datos faltantes:
dfs["GC"].fillna(0, inplace = True)
dfs["Rango_Cons"].fillna("Muy Bajo", inplace = True)
dfs["algun_diag"].fillna(0, inplace = True)
dfs["prop_diag"].fillna(0, inplace = True)
dfs["Marca_Diagnostico"].fillna("Sin Diagnostico", inplace = True)
dfs['GC'] = dfs['GC'].where(dfs['GC'] == 0 , 1)

# Agregamos los parentescos de los hijos:
condiciones = [
    (dfs['Parentesco IdParentesco'] == 3),
    (dfs['Parentesco IdParentesco'] == 4),
    (dfs['Parentesco IdParentesco'] == 5),
    (dfs['Parentesco IdParentesco'] == 6),
    (dfs['Parentesco IdParentesco'] == 7)]
valores_condic = [1,1,1,1,1]
dfs['Marca Hijo'] = np.select(condiciones, valores_condic, default=0) # se analizaron y surgen de consumo 3 == 0

In [25]:
## SI QUEREMOS EXPORTAR POR PERSONA - PERIODO
# Exportación de archivo sin hacer td:
# dfs.to_csv(archivo_salida, encoding = 'latin1', decimal=',', index = False)


## SI QUEREMOS EXPORTAR POR APERTURAS:
segmentacion = ([ 'Periodo', 'Posicion Asociado ID', 'Plan Agrupado', 'Sexo', 'Rango_Edad','Rango_Cons'])

# Defino las métricas a calcular
metricas = ({   'Stock':np.sum,
                'Edad':np.sum,
                'ACE':np.sum,
                'Oncologico':np.sum,
                'Cronico':np.sum,
                'PMI':np.sum,
                'Hepatitis':np.sum,
                'Diabetes':np.sum,
                'GC':np.sum,
                'algun_diag':np.sum,
                'Marca Hijo':np.sum,
            })


td = pd.pivot_table(dfs, index = segmentacion, aggfunc = metricas)
base_asociados = pd.DataFrame(td)
# Exporto la información en csv:
base_asociados.to_csv(archivo_salida , encoding = 'latin1', decimal=',', index = False)

# dfcp = pd.read_csv('./PerPrest-Persona-Consumo-Bnknn_v3.csv', encoding = 'latin1', decimal=',')
# dff = pd.merge(left = dfs, right = dfcp, left_on=("Persona","Periodo"), right_on=("Persona","Periodo_Prest"), how = "left")
# dff = dff.fillna(0)